<a href="https://colab.research.google.com/github/Jade-ChiaChunHou/Computer_Vision/blob/master/%E8%82%A1%E6%B5%B7%E5%B0%8F%E8%8B%B1%E9%9B%84_%E8%82%A1%E5%B8%82%E6%8A%80%E8%A1%93%E5%88%86%E6%9E%90%E7%B3%BB%E5%88%97/%5BColab%5DK%E7%B7%9A%E5%9E%8B%E6%85%8B%E5%88%86%E6%9E%90_%E9%8E%9A%E5%AD%90%26%E5%90%8A%E4%BA%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# K線 反轉型態 - 鎚子 & 吊人

# Part 0: Import 套件
- TA-Lib: https://mrjbq7.github.io/ta-lib/install.html
- plotly: https://plotly.com/python/

In [ ]:
# install ta-lib in colab
url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h516909a_0.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py37ha21ca33_2.tar.bz2'
!curl -L $url | tar xj -C /usr/local/lib/python3.7/dist-packages/ lib/python3.7/site-packages/talib --strip-components=3

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3829    0  3829    0     0   3493      0 --:--:--  0:00:01 --:--:--  3493
100  503k  100  503k    0     0   177k      0  0:00:02  0:00:02 --:--:--  324k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3821    0  3821    0     0  11509      0 --:--:-- --:--:-- --:--:-- 11474
100  406k  100  406k    0     0   194k      0  0:00:02  0:00:02 --:--:--  439k


In [ ]:
#財經分析套件
import talib

# 下載資料套件
import requests as r

#資料處理套件
import pandas as pd
import json
from datetime import datetime, date

# 畫圖套件
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Part 1: 在Ｋ線圖上，標示出 鎚子/吊人 Ｋ線型態
##1.1 下載多日大盤走勢資料
- 股海小英雄youtube影片：[【Python 爬蟲】台股大盤資料免費下載](https://youtu.be/taiSnAeTs7Y)
- 資料來源：[證交所 發行量加權股價指數歷史資料](https://www.twse.com.tw/zh/page/trading/indices/MI_5MINS_HIST.html
)

In [ ]:
def get_stock_market_data(start_year, start_month, end_year, end_month):
    start_date = str(date(start_year, start_month, 1))
    end_date = str(date(end_year, end_month, 1))
    month_list = pd.date_range(start_date, end_date, freq='MS').strftime("%Y%m%d").tolist()
    
    # 下載大盤資料
    df = pd.DataFrame()
    for month in month_list:
        url = "https://www.twse.com.tw/indicesReport/MI_5MINS_HIST?response=json&date=" + month
        res = r.get(url)
        stock_json = res.json()
        stock_df = pd.DataFrame.from_dict(stock_json['data'])
        df = df.append(stock_df, ignore_index = True)
    
    # 資料轉型
    for col in range(0, 5):
        for row in range(df.shape[0]):
            # 把"日期"從字串(string)換成時間(datetime)，並將民國年換成西元年
            if col == 0:
                day = df.iloc[row,0].split('/')
                df.iloc[row, 0] = datetime(int(day[0]) + 1911, int(day[1]), int(day[2]))  
            # 把"開盤價", "最高價", "最低價", "收盤價"帶有逗號的字串(string)換成浮點數(float) 
            elif col != 0:
                df.iloc[row, col] = float(df.iloc[row,col].replace(',', ''))
    
    # 把日期從字串(string)換成時間(datetime)，並將民國年換成西元年
    df.columns = ['日期', '開盤價', '最高價', '最低價', '收盤價']
    return df

In [ ]:
stock = get_stock_market_data(start_year = 2020, start_month = 2, end_year = 2020, end_month = 5)
stock.head(10)

,日期,開盤價,最高價,最低價,收盤價
0,2020-02-03 00:00:00,11365.9,11365.9,11138,11354.9
1,2020-02-04 00:00:00,11399.5,11581.4,11393,11555.9
2,2020-02-05 00:00:00,11601.5,11620.4,11512.7,11573.6
3,2020-02-06 00:00:00,11605.1,11749.7,11605.1,11749.7
4,2020-02-07 00:00:00,11712.7,11712.7,11592.1,11612.8
5,2020-02-10 00:00:00,11514.7,11598,11423.3,11574.1
6,2020-02-11 00:00:00,11614.2,11678.6,11614.2,11664
7,2020-02-12 00:00:00,11693.4,11801.5,11693.4,11774.2
8,2020-02-13 00:00:00,11813.5,11855,11784.9,11791.8
9,2020-02-14 00:00:00,11806.5,11840.8,11788.9,11815.7


## 1.2: 用Ta-Lib套件，找出 鎚子/吊人 K線型態
- [Ta-Lib K線型態介紹](https://www.quantshare.com/index.php?option=manual&dir=/QuantShare%20Language/Candlestick%20Pattern)

### Ta-Lib支援的K線型態們

In [ ]:
print(talib.get_function_groups()['Pattern Recognition'])

['CDL2CROWS', 'CDL3BLACKCROWS', 'CDL3INSIDE', 'CDL3LINESTRIKE', 'CDL3OUTSIDE', 'CDL3STARSINSOUTH', 'CDL3WHITESOLDIERS', 'CDLABANDONEDBABY', 'CDLADVANCEBLOCK', 'CDLBELTHOLD', 'CDLBREAKAWAY', 'CDLCLOSINGMARUBOZU', 'CDLCONCEALBABYSWALL', 'CDLCOUNTERATTACK', 'CDLDARKCLOUDCOVER', 'CDLDOJI', 'CDLDOJISTAR', 'CDLDRAGONFLYDOJI', 'CDLENGULFING', 'CDLEVENINGDOJISTAR', 'CDLEVENINGSTAR', 'CDLGAPSIDESIDEWHITE', 'CDLGRAVESTONEDOJI', 'CDLHAMMER', 'CDLHANGINGMAN', 'CDLHARAMI', 'CDLHARAMICROSS', 'CDLHIGHWAVE', 'CDLHIKKAKE', 'CDLHIKKAKEMOD', 'CDLHOMINGPIGEON', 'CDLIDENTICAL3CROWS', 'CDLINNECK', 'CDLINVERTEDHAMMER', 'CDLKICKING', 'CDLKICKINGBYLENGTH', 'CDLLADDERBOTTOM', 'CDLLONGLEGGEDDOJI', 'CDLLONGLINE', 'CDLMARUBOZU', 'CDLMATCHINGLOW', 'CDLMATHOLD', 'CDLMORNINGDOJISTAR', 'CDLMORNINGSTAR', 'CDLONNECK', 'CDLPIERCING', 'CDLRICKSHAWMAN', 'CDLRISEFALL3METHODS', 'CDLSEPARATINGLINES', 'CDLSHOOTINGSTAR', 'CDLSHORTLINE', 'CDLSPINNINGTOP', 'CDLSTALLEDPATTERN', 'CDLSTICKSANDWICH', 'CDLTAKURI', 'CDLTASUKIGAP', 'CDL

### Ta-Lib K線型態函式名稱
- 鎚子: 'CDLHAMMER'
- 吊人: 'CDLHANGINGMAN'

### Ta-Lib K線型態函式回傳值
- 偏上漲型態: +100
- 無此型態: 0
- 偏下跌型態: -100

### 計算 鎚子＆吊人型態

In [ ]:
df = stock
candle_names = ['CDLHAMMER', 'CDLHANGINGMAN']

# 加各種K線型態的欄位 create columns for  pattern
for candle in candle_names:
    df[candle] = getattr(talib, candle)(df['開盤價'], df['最高價'], df['最低價'], df['收盤價'])

df.head()

,日期,開盤價,最高價,最低價,收盤價,CDLHAMMER,CDLHANGINGMAN
0,2020-02-03 00:00:00,11365.9,11365.9,11138,11354.9,0,0
1,2020-02-04 00:00:00,11399.5,11581.4,11393,11555.9,0,0
2,2020-02-05 00:00:00,11601.5,11620.4,11512.7,11573.6,0,0
3,2020-02-06 00:00:00,11605.1,11749.7,11605.1,11749.7,0,0
4,2020-02-07 00:00:00,11712.7,11712.7,11592.1,11612.8,0,0


### 找出 鎚子: 'CDLHAMMER' 型態¶
- Ta-Lib 函式回傳值: 偏上漲型態 +100

In [ ]:
hammer_idx = df[df['CDLHAMMER']!= 0].index
df[df['CDLHAMMER']!= 0]

,日期,開盤價,最高價,最低價,收盤價,CDLHAMMER,CDLHANGINGMAN
34,2020-03-23 00:00:00,9025.55,9029.52,8750.14,8890.03,100,0
39,2020-03-30 00:00:00,9571.22,9655.7,9415.52,9629.43,100,0
54,2020-04-22 00:00:00,10256.1,10310.9,10140.1,10307.7,100,0
61,2020-05-04 00:00:00,10781.5,10781.5,10658.5,10720.5,100,0
76,2020-05-25 00:00:00,10812.9,10874.5,10719.2,10871.2,100,0


### 找出 吊人: 'CDLHANGINGMAN' 型態
- Ta-Lib 函式回傳值: 偏下跌型態 -100

In [ ]:
hanging_man_idx = df[df['CDLHANGINGMAN']!= 0].index
df[df['CDLHANGINGMAN']!= 0]

,日期,開盤價,最高價,最低價,收盤價,CDLHAMMER,CDLHANGINGMAN
21,2020-03-04 00:00:00,11368.2,11392.4,11297.8,11392.4,0,-100
37,2020-03-26 00:00:00,9667.14,9739.73,9565.02,9736.36,0,-100
58,2020-04-28 00:00:00,10580.2,10620.5,10537.7,10616.1,0,-100


## 1.3: 在Ｋ線圖上，標示出特定Ｋ線型態
- 股海小英雄youtube影片：[【股市分析】用K線預測大盤趨勢](https://youtu.be/FX6YAyOdEaE)

In [ ]:
# 畫大盤Ｋ線圖
fig = go.Figure(data = [go.Candlestick(x = df['日期'],
                open = df['開盤價'],
                high = df['最高價'],
                low = df['最低價'],
                close = df['收盤價'],
                increasing_line_color = 'red', 
                decreasing_line_color = 'green')])
# 標示鎚子型態
for idx in range(len(hammer_idx)):
    day = df.iloc[hammer_idx[idx], 0]
    high = float(df[df['日期'] == day]['最高價'])
    fig.add_annotation(x = day, 
                       y = high + 10,
                       opacity = 0.9,
                       text = "鎚子 " + str(idx + 1),
                       showarrow = True,
                       arrowhead = 1)
# 標示吊人型態
for idx in range(len(hanging_man_idx)):
    day = df.iloc[hanging_man_idx[idx], 0]
    high = float(df[df['日期'] == day]['最高價'])
    fig.add_annotation(x = day, 
                       y = high + 10,
                       text = "吊人" + str(idx + 1),
                       font_color = 'blue',
                       showarrow = True,
                       arrowhead = 1,
                       arrowcolor = 'blue',
                       opacity = 0.5)
# 設圖標及圖長寬
fig.update_layout(
    title_text = "2020/02 - 2020/05 大盤K線圖 - 鎚子(Hammer)型態",
    width = 1200,
    height = 600
)

# 設x軸標題
fig.update_xaxes(title_text = "日期")

# 設y軸標題
fig.update_yaxes(title_text = "指數")

fig.show()

# Part 2: 畫出特定型態(如：鎚子)的當日走勢圖


## 2.1 下載大盤當日每五秒資訊
- 股海小英雄youtube影片：[【Python 爬蟲】台股各類指數免費下載](https://youtu.be/wewSSlgJt_I)
- 資料來源：[證交所每5秒指數盤後統計](https://www.twse.com.tw/zh/page/trading/exchange/MI_5MINS_INDEX.html
)

In [ ]:
def get_stock_daily(year, month, day):
    stock_date = str(date(year, month, day).strftime("%Y%m%d"))    
    
    # 下載盤後每5秒統計資料
    df = pd.DataFrame()
    url = "https://www.twse.com.tw/exchangeReport/MI_5MINS_INDEX?response=json&date=" + stock_date
    res = r.get(url)
    stock_json = res.json()
    stock_df = pd.DataFrame.from_dict(stock_json['data'])
    df = df.append(stock_df, ignore_index = True) 
    df.columns = stock_json['fields']
    
    # 只取'發行量加權股價指數'
    df = df[['時間', '發行量加權股價指數']]
    
    # 資料轉型：'發行量加權股價指數'從字串轉為浮點數
    for row in range(df.shape[0]):
        df.iloc[row, 1] = float(df.iloc[row,1].replace(',', ''))
    
    return df

In [ ]:
daily_stock = get_stock_daily(2020, 3, 23)
daily_stock.head()

,時間,發行量加權股價指數
0,09:00:00,9234.09
1,09:00:05,9025.55
2,09:00:10,8937.7
3,09:00:15,8904.66
4,09:00:20,8893.29


## 2.2 畫出特定型態(如：鎚子)的日K線及當日走勢圖

In [ ]:
# 設定subplots
fig = make_subplots(
    rows=1, 
    cols=2,
    column_widths=[0.3, 0.7],
    subplot_titles = ('2020/3/23 日Ｋ圖', '2020/3/23 大盤指數'))
    #row_heights=[0.25, 0.05, 0.25, 0.05, 0.4])

# 畫日Ｋ圖
df_d = df[df['日期']== datetime(2020, 3, 23)]
fig.add_trace(
    go.Candlestick(x = df_d['日期'],
                   open = df_d['開盤價'],
                   high = df_d['最高價'],
                   low = df_d['最低價'],
                   close = df_d['收盤價'],
                   increasing_line_color = 'red', 
                   decreasing_line_color = 'green',
                   name = '日K線'),
                   row = 1, 
                   col = 1
)

# 畫當日大盤指數
daily_stock = get_stock_daily(2020, 3, 23)
fig.add_trace(
    go.Scatter(x = daily_stock['時間'], 
               y = daily_stock['發行量加權股價指數'], 
               name = '大盤指數',
               line = dict(color = 'green')),
    row = 1, col = 2,
    
)

# 設圖標及圖長寬
fig.update_layout(
    title_text = "2020/03/23 大盤指數 - 鎚子(Hammer)型態",
    width = 1200, 
    height = 500,
    xaxis_rangeslider_visible = False
)

# 設x軸標題及標示
fig.update_xaxes(
        ticktext=["09:00:00", "11:00:00", "13:00:00"],
        tickvals=["09:00:00", "11:00:00", "13:00:00"]
    )

# 設y軸標題
fig.update_yaxes(title_text="指數")

fig.show()

## 2.3 畫出多日特定型態(如：鎚子)的K線及型態當日走勢圖

In [ ]:
# 設定subplots
fig = make_subplots(
    rows = len(hammer_idx) + 1, 
    cols = 1,
    subplot_titles = ('2020/02 - 2020/05 K線圖', 
                      '2020/3/23 大盤指數', 
                      '2020/3/30 大盤指數', 
                      '2020/4/22 大盤指數', 
                      '2020/5/4 大盤指數',
                      '2020/5/25 大盤指數'))

# 畫Ｋ線圖
df = stock
fig.add_trace(
    go.Candlestick(x = df['日期'],
                   open = df['開盤價'],
                   high = df['最高價'],
                   low = df['最低價'],
                   close = df['收盤價'],
                   increasing_line_color = 'red', 
                   decreasing_line_color = 'green',
                   name = '日K線'),
    row = 1, 
    col = 1,
)

# 標示鎚子型態
idx = hammer_idx
for idx in range(len(hammer_idx)):
    day = df.iloc[hammer_idx[idx], 0]
    high = float(df[df['日期']== day]['最高價'])
    fig.add_annotation(x = day, 
                       y = high + 10,
                       opacity = 0.9,
                       text = "鎚子 " + str(idx + 1),
                       showarrow = True,
                       arrowhead = 1)
    
    fig.update_xaxes(
        ticktext=["09:00:00", "11:00:00", "13:00:00"],
        tickvals=["09:00:00", "11:00:00", "13:00:00"]
    )


# 畫鎚子型態的當日大盤指數
for idx in range(len(hammer_idx)):
    candle_day = df.iloc[hammer_idx[idx]][0]
    daily_stock = get_stock_daily(candle_day.year, candle_day.month, candle_day.day)
    fig.add_trace(
        go.Scatter(x = daily_stock['時間'], 
                   y = daily_stock['發行量加權股價指數'],
                   name = str(df.iloc[hammer_idx[idx]][0]).split(' ')[0]),
        row = idx + 2, 
        col = 1
    )

# 設圖標及圖長寬    
fig.update_layout(
    title_text = "2020/02 - 2020/05 大盤K線圖 - 鎚子(Hammer)型態",
    width = 1200,
    height = 1500,
    xaxis_rangeslider_visible = False
)

fig.show()